In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import tensorflow as tf
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical
from tensorflow.keras.layers import Dense, Embedding, Flatten, BatchNormalization, LayerNormalization, LSTM, SimpleRNN, Dropout, Bidirectional
from tensorflow.keras.models import Sequential, Model
from transformers import BertTokenizer, BertForSequenceClassification, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score

import warnings
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Cell_Phones_and_Accessories_5.json'

In [ ]:
df = pd.read_json(path, lines='TRUE', nrows = 40000)

# EDA

In [ ]:
# Checking the dataframe
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
# Let's work with 'reviewText' to train and predict the overall rating
reviews = pd.DataFrame(df['reviewText'])

In [ ]:
reviews.head()

,reviewText
0,They look good and stick good! I just don't li...
1,These stickers work like the review says they ...
2,These are awesome and make my phone look so st...
3,Item arrived in great time and was in perfect ...
4,"awesome! stays on, and looks great. can be use..."


In [ ]:
reviews.shape

(40000, 1)

In [ ]:
reviews.isnull().sum()

reviewText    0
dtype: int64

In [ ]:
reviews.dtypes

reviewText    object
dtype: object

In [ ]:
# convert all reviews into lower case
reviews['reviewText'] = reviews['reviewText'].str.lower()

In [ ]:
reviews.head()

,reviewText
0,they look good and stick good! i just don't li...
1,these stickers work like the review says they ...
2,these are awesome and make my phone look so st...
3,item arrived in great time and was in perfect ...
4,"awesome! stays on, and looks great. can be use..."


In [ ]:
# Function to remove special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    text = re.sub(r'\d+','',text)
    return text

In [ ]:
reviews['reviewText'] = reviews['reviewText'].apply(remove_special_characters)

In [ ]:
reviews.head()

,reviewText
0,they look good and stick good i just dont like...
1,these stickers work like the review says they ...
2,these are awesome and make my phone look so st...
3,item arrived in great time and was in perfect ...
4,awesome stays on and looks great can be used o...


In [ ]:
# Remove white spaces
def remove_whitespace(text):
    cleaned_text = ' '.join(text.split())
    return cleaned_text

In [ ]:
reviews['reviewText'] = reviews['reviewText'].apply(remove_whitespace)

In [ ]:
reviews.head()

,reviewText
0,they look good and stick good i just dont like...
1,these stickers work like the review says they ...
2,these are awesome and make my phone look so st...
3,item arrived in great time and was in perfect ...
4,awesome stays on and looks great can be used o...


In [ ]:
 # Remove HTML tags using BeautifulSoup
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text()
    return cleaned_text

In [ ]:
reviews['reviewText'] = reviews['reviewText'].apply(remove_html_tags)

<ipython-input-24-2a0f0d36394e>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
reviews.head()

,reviewText
0,they look good and stick good i just dont like...
1,these stickers work like the review says they ...
2,these are awesome and make my phone look so st...
3,item arrived in great time and was in perfect ...
4,awesome stays on and looks great can be used o...


In [ ]:
# remove square brackets
def remove_square_brackets(text):
  text = re.sub('\[[^]]*\]', '', text)
  text = re.sub(r'\d+','',text)
  return text

In [ ]:
reviews['reviewText'] = reviews['reviewText'].apply(remove_square_brackets)

In [ ]:
df['overall'].dtypes

dtype('int64')

In [ ]:
df['overall'] = df['overall'].astype('category')

In [ ]:
print(df['overall'].dtypes)

category


In [ ]:
constraints = ['#B34D22', '#EBE00C', '#1FEB0C', '#0C92EB', '#EB0CD5']

def categorical_variable_summary(df, column_name):
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('Countplot', 'Percentage'),
                        specs=[[{'type': 'xy'}, {'type': 'domain'}]])

    fig.add_trace(go.Bar(y=df[column_name].value_counts().values.tolist(),
                         x=[str(i) for i in df[column_name].value_counts().index],
                         text=df[column_name].value_counts().values.tolist(),
                         textfont=dict(size=14),
                         name=column_name,
                         textposition='auto',
                         showlegend=False,
                         marker=dict(color=constraints, line=dict(color='#DBE6EC', width=1))),
                  row=1, col=1)

    fig.add_trace(go.Pie(labels=df[column_name].value_counts().keys(),
                         values=df[column_name].value_counts().values,
                         textfont=dict(size=18),
                         textposition='auto',
                         showlegend=False,
                         name=column_name,
                         marker=dict(colors=constraints)),
                  row=1, col=2)

    fig.update_layout(title={'text': column_name,
                             'y': 0.9,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white',
                      height=500,  # Adjust the height of the plot (increase as needed)
                      width=800)

    iplot(fig)
    #fig.show()

In [ ]:
categorical_variable_summary(df,'overall')

In [ ]:
# Lets build models

X_train, X_test, y_train, y_test = train_test_split(reviews["reviewText"], df["overall"], test_size = 0.3, random_state = 42,stratify=df["overall"])
X_train[:5]

31846    very good quality and holds necessary cards to...
17729    zagg thumbs downthank god i didnt pay retail f...
5272     these charging cables work excellent thank you...
13045    great usb charger for your car led glows to he...
29185    prosno interference with touch screenless fing...
Name: reviewText, dtype: object

## TFIDF

In [ ]:
tfidf = TfidfVectorizer(max_features=1000,
                                    ngram_range=(1,1),
                                    stop_words= None)

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train)

In [ ]:
X_train_tfidf

<28000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1279821 stored elements in Compressed Sparse Row format>

In [ ]:
X_train_dense = pd.DataFrame(X_train_tfidf.todense())

In [ ]:
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
X_test_dense = pd.DataFrame(X_test_tfidf.todense())

In [ ]:
print(f"Shape of X_train is {X_train_dense.shape}")
print(f"Shape of y_train is {y_train.shape}")
print(f"Shape of X_test is {X_test_dense.shape}")
print(f"Shape of y_test is {y_test.shape}")

Shape of X_train is (28000, 1000)
Shape of y_train is (28000,)
Shape of X_test is (12000, 1000)
Shape of y_test is (12000,)


In [ ]:
##Building Log regression
lr = LogisticRegression()

lr_model = lr.fit(X_train_dense, y_train)

In [ ]:
lr_train_preds = lr_model.predict(X_train_dense)
lr_test_preds = lr_model.predict(X_test_dense)

In [ ]:
print(f"Confusion Matrix: \n\n {confusion_matrix(y_train, lr_train_preds)} \n")
print(f"Test Data: \n\n {classification_report(y_test, lr_test_preds)}")

Confusion Matrix: 

 [[ 1530   118   175   124   629]
 [  395   365   324   270   570]
 [  211   102   997   757  1238]
 [  121    54   355  2040  3185]
 [  143    49   231   892 13125]] 

Test Data: 

               precision    recall  f1-score   support

           1       0.58      0.55      0.57      1104
           2       0.29      0.10      0.14       824
           3       0.37      0.22      0.28      1416
           4       0.43      0.30      0.35      2467
           5       0.69      0.90      0.78      6189

    accuracy                           0.61     12000
   macro avg       0.47      0.41      0.42     12000
weighted avg       0.56      0.61      0.57     12000



In [ ]:
# Pipeline

# Naive Bayes
text_clf_nb = Pipeline([('tfidf',
                         TfidfVectorizer(ngram_range=(1,1),
                                         max_features=12000,
                                         max_df=0.95,
                                         stop_words=None)),
('nb_clf',
 MultinomialNB()),
])

# SVC
text_clf_lsvc = Pipeline([('tfidf',
                           TfidfVectorizer(ngram_range=(1,1),
                                           max_features=12000,
                                           max_df=0.95,
                                           stop_words=None)),
('clf', LinearSVC()),
])


# Random Forest

text_clf_RF = Pipeline([('tfidf',
                         TfidfVectorizer(ngram_range=(1,1),
                                         max_features=900,
                                         max_df=0.95,
                                         stop_words=None)),

('Random Forest',
 RandomForestClassifier(n_estimators=100,
                        criterion='gini',
                         max_depth=50))
])

In [ ]:
def models_run(model,modelname):

  preds = model.predict(X_test)

  print(f"Confusion Matrix for {modelname} \n")
  print(metrics.confusion_matrix(y_test,preds))
  print('\n')

  print(f"Classification Report for {modelname}\n")
  print(metrics.classification_report(y_test,preds))


  print(f"Accuracy for: {modelname}")
  print(metrics.accuracy_score(y_test,preds).round(2))

In [ ]:
#Naive bayes
text_clf_nb.fit(X_train, y_train)
models_run(text_clf_nb, "Naive Bayes")

Confusion Matrix for Naive Bayes 

[[  31    0    2   11 1060]
 [   3    0    0   37  784]
 [   2    0    2   52 1360]
 [   0    0    1   86 2380]
 [   0    0    0   42 6147]]


Classification Report for Naive Bayes

              precision    recall  f1-score   support

           1       0.86      0.03      0.05      1104
           2       0.00      0.00      0.00       824
           3       0.40      0.00      0.00      1416
           4       0.38      0.03      0.06      2467
           5       0.52      0.99      0.69      6189

    accuracy                           0.52     12000
   macro avg       0.43      0.21      0.16     12000
weighted avg       0.47      0.52      0.37     12000

Accuracy for: Naive Bayes
0.52


In [ ]:
#SVC
text_clf_lsvc.fit(X_train, y_train)
models_run(text_clf_lsvc, "SVC")

Confusion Matrix for SVC 

[[ 635   91   96   74  208]
 [ 238   97  163  137  189]
 [ 129   96  312  408  471]
 [  64   51  194  770 1388]
 [  68   33  150  554 5384]]


Classification Report for SVC

              precision    recall  f1-score   support

           1       0.56      0.58      0.57      1104
           2       0.26      0.12      0.16       824
           3       0.34      0.22      0.27      1416
           4       0.40      0.31      0.35      2467
           5       0.70      0.87      0.78      6189

    accuracy                           0.60     12000
   macro avg       0.45      0.42      0.43     12000
weighted avg       0.55      0.60      0.57     12000

Accuracy for: SVC
0.6


In [ ]:
#Random Forest
text_clf_RF.fit(X_train, y_train)
models_run(text_clf_RF, "Random Forest")

Confusion Matrix for Random Forest 

[[ 234    0   20   58  792]
 [  64    2   29   65  664]
 [  36    1   79  152 1148]
 [  14    1   30  252 2170]
 [  11    0   13  112 6053]]


Classification Report for Random Forest

              precision    recall  f1-score   support

           1       0.65      0.21      0.32      1104
           2       0.50      0.00      0.00       824
           3       0.46      0.06      0.10      1416
           4       0.39      0.10      0.16      2467
           5       0.56      0.98      0.71      6189

    accuracy                           0.55     12000
   macro avg       0.51      0.27      0.26     12000
weighted avg       0.52      0.55      0.44     12000

Accuracy for: Random Forest
0.55


**Word Embedding**

In [ ]:
t = Tokenizer(num_words=1500, oov_token= 'unk')

In [ ]:
t.fit_on_texts(reviews['reviewText'])

In [ ]:
vocab_size = len(t.word_index)+1
vocab_size

68116

In [ ]:
review_sequence = t.texts_to_sequences(reviews['reviewText'])

In [ ]:
max_length = 40

In [ ]:
review_padded = pad_sequences(review_sequence, maxlen= max_length, padding='post')

In [ ]:
review_padded.shape

(40000, 40)

In [ ]:
le =LabelEncoder()
y_target = le.fit_transform(df["overall"])

In [ ]:
labels = to_categorical(y_target,num_classes =5)

In [ ]:
model = Sequential()

model.add(Embedding(input_length=max_length,
                    input_dim=vocab_size,
                    output_dim=40))

model.add(LayerNormalization())
model.add(Flatten())

model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 40)            2724640   
                                                                 
 layer_normalization_1 (Laye  (None, 40, 40)           80        
 rNormalization)                                                 
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 8005      
                                                                 
Total params: 2,732,725
Trainable params: 2,732,725
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.fit(review_padded,
          labels,
          epochs=7,
          verbose=1,
          steps_per_epoch = 50,
          validation_split=0.25,
          shuffle=True,
          callbacks=[early_stopping])

Epoch 1/7
50/50 [==============================] - 15s 176ms/step - loss: 1.4540 - accuracy: 0.4554 - val_loss: 1.2766 - val_accuracy: 0.5055
Epoch 2/7
50/50 [==============================] - 5s 95ms/step - loss: 1.0983 - accuracy: 0.5705 - val_loss: 1.1697 - val_accuracy: 0.5390
Epoch 3/7
50/50 [==============================] - 5s 101ms/step - loss: 0.9413 - accuracy: 0.6367 - val_loss: 1.1415 - val_accuracy: 0.5439
Epoch 4/7
50/50 [==============================] - 1s 25ms/step - loss: 0.8409 - accuracy: 0.6826 - val_loss: 1.1560 - val_accuracy: 0.5456
Epoch 5/7
50/50 [==============================] - 1s 25ms/step - loss: 0.7576 - accuracy: 0.7225 - val_loss: 1.1926 - val_accuracy: 0.5446
Epoch 6/7
50/50 [==============================] - 1s 24ms/step - loss: 0.6839 - accuracy: 0.7552 - val_loss: 1.2352 - val_accuracy: 0.5313


## Glove Embedding

In [ ]:
reviews = list(reviews['reviewText'])
reviews[0:1]

['they look good and stick good i just dont like the rounded shape because i was always bumping it and siri kept popping up and it was irritating i just wont buy a product like this again']

In [ ]:
t = Tokenizer(num_words=7000, lower=True)

In [ ]:
train_text = reviews[:len(df) - 10000]
test_text = reviews[-10000:]

In [ ]:
print(f"The whole length of the Text is {len(reviews)}")
print(f"The Train's length is {len(train_text)}")
print(f"The Test's length is {len(test_text)}")

The whole length of the Text is 40000
The Train's length is 30000
The Test's length is 10000


In [ ]:
t.fit_on_texts(train_text)

In [ ]:
t.word_index
len(t.word_index)

57601

In [ ]:
embeddings_index = dict()

glove = open("/content/drive/MyDrive/glove.6B.100d.txt", 'r', encoding = 'utf8')

for line in glove:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  embeddings_index[word] = coefs
glove.close()

In [ ]:
vocab = len(t.word_index)+1
print(vocab)

57602


In [ ]:
embedding_matrix = np.zeros((vocab, 100))

for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

print(f"The shape is {embedding_matrix.shape}")

The shape is (57602, 100)


## Model Building

1. LSTM with GloVe embeddings

In [ ]:
train_indices = t.texts_to_sequences(train_text)

max_length = 100

train_indices = pad_sequences(train_indices,
                              maxlen = max_length,
                              padding = 'post')

In [ ]:
model1 = Sequential()

model1.add(Embedding(input_dim = vocab,
                     output_dim = 100,
                     input_length = max_length,
                     weights = [embedding_matrix],
                     trainable = False))

model1.add(LSTM(34))

model1.add(Dense(5,activation = 'softmax'))
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          5760200   
                                                                 
 lstm (LSTM)                 (None, 34)                18360     
                                                                 
 dense_2 (Dense)             (None, 5)                 175       
                                                                 
Total params: 5,778,735
Trainable params: 18,535
Non-trainable params: 5,760,200
_________________________________________________________________


In [ ]:
model1.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['acc'])

In [ ]:
train_labels = labels[:len(labels) - 10000]
test_labels = labels[-10000:]

print(f"The size of train_labels: {len(train_labels)}")
print(f"The size of test_labels: {len(test_labels)}")

The size of train_labels: 30000
The size of test_labels: 10000


In [ ]:
train_indices = np.asarray(train_indices)
train_y = np.asarray(train_labels)

In [ ]:
model1.fit(train_indices,
           train_y,
           batch_size = 32,
           validation_split = 0.25,
           epochs=7,
           steps_per_epoch = 50,
           verbose=1)

Epoch 1/7
50/50 [==============================] - 5s 34ms/step - loss: 1.4466 - acc: 0.5069 - val_loss: 1.3186 - val_acc: 0.5181
Epoch 2/7
50/50 [==============================] - 2s 33ms/step - loss: 1.3146 - acc: 0.5250 - val_loss: 1.3094 - val_acc: 0.5173
Epoch 3/7
50/50 [==============================] - 2s 32ms/step - loss: 1.3486 - acc: 0.4875 - val_loss: 1.3076 - val_acc: 0.5176
Epoch 4/7
50/50 [==============================] - 2s 36ms/step - loss: 1.3157 - acc: 0.5200 - val_loss: 1.3095 - val_acc: 0.5179
Epoch 5/7
50/50 [==============================] - 2s 36ms/step - loss: 1.3435 - acc: 0.5113 - val_loss: 1.3143 - val_acc: 0.5181
Epoch 6/7
50/50 [==============================] - 1s 22ms/step - loss: 1.3205 - acc: 0.5188 - val_loss: 1.3086 - val_acc: 0.5180
Epoch 7/7
50/50 [==============================] - 1s 23ms/step - loss: 1.2970 - acc: 0.5356 - val_loss: 1.3114 - val_acc: 0.5181


In [ ]:
test_indices = t.texts_to_sequences(test_text)
test_indices = pad_sequences(test_indices, maxlen = max_length, padding = 'post')
test_indices = np.asarray(test_indices)
test_y = np.asarray(test_labels)

In [ ]:
scores = model1.evaluate(test_indices, test_y, verbose=0)
loss = scores[0]
accuracy = scores[1]

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Loss: 1.3163
Test Accuracy: 51.26%


In [ ]:
## Stacked LSTM

model2= Sequential()

model2.add(Embedding(input_dim = vocab,
                     output_dim = 100,
                     input_length = max_length,
                     weights = [embedding_matrix],
                     trainable = False))

model2.add(Dropout(0.2))

model2.add(Bidirectional(LSTM(32, return_sequences=True)))

model2.add(LayerNormalization())

model2.add(Bidirectional(LSTM(64)))

model2.add(Dropout(0.2))

model2.add(Dense(5, activation='softmax'))

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          5760200   
                                                                 
 dropout (Dropout)           (None, 100, 100)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 100, 64)          34048     
 l)                                                              
                                                                 
 layer_normalization_2 (Laye  (None, 100, 64)          128       
 rNormalization)                                                 
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                      

In [ ]:
model2.fit(train_indices,
           train_y,
           batch_size = 32,
           validation_split = 0.25,
           epochs=7,
           steps_per_epoch = 50,
           callbacks=[early_stopping],
           verbose=1)

Epoch 1/7
50/50 [==============================] - 13s 101ms/step - loss: 1.3931 - accuracy: 0.4762 - val_loss: 1.3052 - val_accuracy: 0.5181
Epoch 2/7
50/50 [==============================] - 3s 71ms/step - loss: 1.3306 - accuracy: 0.5119 - val_loss: 1.2962 - val_accuracy: 0.5180
Epoch 3/7
50/50 [==============================] - 3s 54ms/step - loss: 1.3245 - accuracy: 0.5063 - val_loss: 1.2728 - val_accuracy: 0.5181
Epoch 4/7
50/50 [==============================] - 3s 70ms/step - loss: 1.3063 - accuracy: 0.5138 - val_loss: 1.2554 - val_accuracy: 0.5187
Epoch 5/7
50/50 [==============================] - 3s 52ms/step - loss: 1.2559 - accuracy: 0.5256 - val_loss: 1.2330 - val_accuracy: 0.5177
Epoch 6/7
50/50 [==============================] - 3s 70ms/step - loss: 1.2739 - accuracy: 0.5119 - val_loss: 1.2397 - val_accuracy: 0.5181
Epoch 7/7
50/50 [==============================] - 4s 78ms/step - loss: 1.2611 - accuracy: 0.5138 - val_loss: 1.2583 - val_accuracy: 0.5204


In [ ]:
test_indices = t.texts_to_sequences(test_text)
test_indices = pad_sequences(test_indices, maxlen = max_length, padding = 'post')
test_indices = np.asarray(test_indices)
test_y = np.asarray(test_labels)

In [ ]:
scores = model2.evaluate(test_indices, test_y, verbose=0)
loss = scores[0]
accuracy = scores[1]

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Loss: 1.2560
Test Accuracy: 51.64%


## Another approach with target attribute

In [ ]:
data = pd.read_json('/content/drive/MyDrive/Cell_Phones_and_Accessories_5.json', lines='TRUE', nrows = 40000)

In [ ]:
data['rating'] = data.overall.replace({
    1:'negative',
    2:'negative',
    3:'neutral',
    4:'positive',
    5:'positive'
})

In [ ]:
data['rating'].value_counts()

positive    28851
negative     6428
neutral      4721
Name: rating, dtype: int64

In [ ]:
X = data['summary']
y = data['rating']

In [ ]:
print(X.dtypes)

object


In [ ]:
print(y.dtypes)

object


In [ ]:
X = X.astype(str)

In [ ]:
y = y.astype('category')

In [ ]:
# creating functions for text processing
import string
string.punctuation
def final(X_data_full):

    # function for removing punctuations
    def remove_punct(X_data_func):
        string1 = X_data_func.lower()
        translation_table = dict.fromkeys(map(ord, string.punctuation),' ')
        string2 = string1.translate(translation_table)
        return string2

    X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct(X_data_full[i])
        X_data_full_clear_punct.append(test_data)

    # function to remove stopwords
    def remove_stopwords(X_data_func):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        string2 = pattern.sub(' ', X_data_func)
        return string2

    X_data_full_clear_stopwords = []
    for i in range(len(X_data_full)):
        test_data = remove_stopwords(X_data_full[i])
        X_data_full_clear_stopwords.append(test_data)

    # function for tokenizing
    def tokenize_words(X_data_func):
        words = nltk.word_tokenize(X_data_func)
        return words

    X_data_full_tokenized_words = []
    for i in range(len(X_data_full)):
        test_data = tokenize_words(X_data_full[i])
        X_data_full_tokenized_words.append(test_data)

    # function for lemmatizing
    lemmatizer = WordNetLemmatizer()
    def lemmatize_words(X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words

    X_data_full_lemmatized_words = []
    for i in range(len(X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        X_data_full_lemmatized_words.append(test_data)



    # creating the bag of words model
    cv = CountVectorizer(max_features=1000)
    X_data_full_vector = cv.fit_transform(X_data_full_lemmatized_words).toarray()


    tfidf = TfidfTransformer()
    X_data_full_tfidf = tfidf.fit_transform(X_data_full_vector).toarray()

    return X_data_full_tfidf

In [ ]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
X = final(X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

In [ ]:
NB_clf = MultinomialNB()
NB_clf.fit(X_train, y_train)
predictions = NB_clf.predict(X_test)

print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

NB_accuracy = round((accuracy_score(y_test, predictions)*100),2)

print("Accuracy : " , NB_accuracy , " %")

              precision    recall  f1-score   support

    negative       0.72      0.44      0.54      1904
     neutral       0.60      0.16      0.26      1371
    positive       0.81      0.97      0.88      8725

    accuracy                           0.79     12000
   macro avg       0.71      0.52      0.56     12000
weighted avg       0.77      0.79      0.76     12000

[[ 829   73 1002]
 [ 151  225  995]
 [ 164   80 8481]]
Accuracy :  79.46  %


In [ ]:
rfc=RandomForestClassifier(n_estimators= 50, random_state= 99)
rfc.fit(X_train, y_train)
predictions = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

rfc_accuracy = round((accuracy_score(y_test, predictions) * 100), 2)

print("Accuracy : " , rfc_accuracy , " %")

              precision    recall  f1-score   support

    negative       0.70      0.59      0.64      1904
     neutral       0.46      0.27      0.34      1371
    positive       0.86      0.94      0.90      8725

    accuracy                           0.81     12000
   macro avg       0.67      0.60      0.63     12000
weighted avg       0.79      0.81      0.80     12000

[[1130  183  591]
 [ 251  376  744]
 [ 238  260 8227]]
Accuracy :  81.11  %


In [ ]:
y_train_dummies = pd.get_dummies(y_train).values
print('Shape of Label: ', y_train_dummies.shape)

Shape of Label:  (28000, 3)


1. LSTM

In [ ]:
embedding_size=32
max_words=5000

model = Sequential()
model.add(Embedding(max_words, embedding_size, input_length=X_train.shape[1]))
model.add(LSTM(100))
model.add(Dense(3,activation='softmax'))

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1000, 32)          160000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dense_4 (Dense)             (None, 3)                 303       
                                                                 
Total params: 213,503
Trainable params: 213,503
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train_dummies, epochs=5, validation_split = 0.25, steps_per_epoch = 50, batch_size=50, verbose=1)

Epoch 1/5
50/50 [==============================] - 7s 98ms/step - loss: 0.8466 - accuracy: 0.7052 - val_loss: 0.7890 - val_accuracy: 0.7176
Epoch 2/5
50/50 [==============================] - 4s 80ms/step - loss: 0.7757 - accuracy: 0.7260 - val_loss: 0.7913 - val_accuracy: 0.7176
Epoch 3/5
50/50 [==============================] - 4s 86ms/step - loss: 0.8074 - accuracy: 0.7080 - val_loss: 0.8086 - val_accuracy: 0.7176
Epoch 4/5
50/50 [==============================] - 4s 86ms/step - loss: 0.7948 - accuracy: 0.7140 - val_loss: 0.7960 - val_accuracy: 0.7176
Epoch 5/5
50/50 [==============================] - 4s 90ms/step - loss: 0.7821 - accuracy: 0.7292 - val_loss: 0.7896 - val_accuracy: 0.7176


In [ ]:
y_test_dummies = pd.get_dummies(y_test).values
print('Shape of Label: ', y_test_dummies.shape)

Shape of Label:  (12000, 3)


In [ ]:
# Evaluate the model on the test data
scores = model.evaluate(X_test, y_test_dummies, verbose=0)

# The 'scores' variable will contain the evaluation results, which typically include loss and accuracy
loss = scores[0]
accuracy = scores[1]

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Loss: 0.7753
Test Accuracy: 72.71%


2. BIDIRECTIONAL LSTM

In [ ]:
model2= Sequential()

model2.add(Embedding(input_dim=vocab,output_dim=100,input_length=X_train.shape[1]))

model2.add(Dropout(0.2))

model2.add(Bidirectional(LSTM(32, return_sequences=True)))

model2.add(LayerNormalization())

model2.add(Bidirectional(LSTM(64)))

model2.add(Dropout(0.2))

model2.add(Dense(3, activation='softmax'))

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1000, 100)         5760200   
                                                                 
 dropout_2 (Dropout)         (None, 1000, 100)         0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 1000, 64)         34048     
 nal)                                                            
                                                                 
 layer_normalization_3 (Laye  (None, 1000, 64)         128       
 rNormalization)                                                 
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                      

In [ ]:
model2.fit(X_train, y_train_dummies, batch_size=60, epochs=7, steps_per_epoch = 20, verbose=1, validation_split=0.25, shuffle=True)

Epoch 1/7
20/20 [==============================] - 17s 457ms/step - loss: 0.8401 - accuracy: 0.6850 - val_loss: 0.8310 - val_accuracy: 0.7176
Epoch 2/7
20/20 [==============================] - 13s 665ms/step - loss: 0.7741 - accuracy: 0.7358 - val_loss: 0.7885 - val_accuracy: 0.7176
Epoch 3/7
20/20 [==============================] - 13s 648ms/step - loss: 0.7870 - accuracy: 0.7283 - val_loss: 0.7896 - val_accuracy: 0.7176
Epoch 4/7
20/20 [==============================] - 13s 648ms/step - loss: 0.7812 - accuracy: 0.7267 - val_loss: 0.7887 - val_accuracy: 0.7176
Epoch 5/7
20/20 [==============================] - 7s 371ms/step - loss: 0.7864 - accuracy: 0.7308 - val_loss: 0.7929 - val_accuracy: 0.7176
Epoch 6/7
20/20 [==============================] - 13s 670ms/step - loss: 0.8098 - accuracy: 0.7100 - val_loss: 0.7886 - val_accuracy: 0.7176
Epoch 7/7
20/20 [==============================] - 13s 653ms/step - loss: 0.7863 - accuracy: 0.7225 - val_loss: 0.7912 - val_accuracy: 0.7176


In [ ]:
# Evaluate the model on the test data
scores = model2.evaluate(X_test, y_test_dummies, verbose=0)

# The 'scores' variable will contain the evaluation results, which typically include loss and accuracy
loss = scores[0]
accuracy = scores[1]

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Loss: 0.7749
Test Accuracy: 72.71%


3. RNN WITH GLOVE EMBEDDING

In [ ]:
model3 = Sequential()

model3.add(Embedding(input_dim = vocab,output_dim = 100,input_length=X_train.shape[1],weights = [embedding_matrix],trainable = False))

model3.add(Dropout(0.2))

model3.add(SimpleRNN(50))

model3.add(BatchNormalization())

model3.add(Dropout(0.2))

model3.add(Dense(3,activation = 'softmax'))


In [ ]:
model3.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['acc'])
model3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 1000, 100)         5760200   
                                                                 
 dropout_4 (Dropout)         (None, 1000, 100)         0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 50)                7550      
                                                                 
 batch_normalization (BatchN  (None, 50)               200       
 ormalization)                                                   
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 3)                 153       
                                                      

In [ ]:
model3.fit(X_train, y_train_dummies, epochs=7, batch_size=50, validation_split = 0.3, steps_per_epoch=30, verbose=1,callbacks=[early_stopping])

Epoch 1/7
30/30 [==============================] - 50s 2s/step - loss: 1.1017 - acc: 0.6233 - val_loss: 0.9789 - val_acc: 0.7175
Epoch 2/7
30/30 [==============================] - 37s 1s/step - loss: 0.9448 - acc: 0.7347 - val_loss: 0.9225 - val_acc: 0.7186
Epoch 3/7
30/30 [==============================] - 36s 1s/step - loss: 0.8930 - acc: 0.7107 - val_loss: 0.8679 - val_acc: 0.7186
Epoch 4/7
30/30 [==============================] - 36s 1s/step - loss: 0.8124 - acc: 0.7273 - val_loss: 0.8345 - val_acc: 0.7186
Epoch 5/7
30/30 [==============================] - 36s 1s/step - loss: 0.8093 - acc: 0.7100 - val_loss: 0.8146 - val_acc: 0.7186
Epoch 6/7
30/30 [==============================] - 38s 1s/step - loss: 0.7837 - acc: 0.7220 - val_loss: 0.8038 - val_acc: 0.7186
Epoch 7/7
30/30 [==============================] - 36s 1s/step - loss: 0.7844 - acc: 0.7227 - val_loss: 0.7962 - val_acc: 0.7186


In [ ]:
# Evaluate the model on the test data
scores = model3.evaluate(X_test, y_test_dummies, verbose=0)

# The 'scores' variable will contain the evaluation results, which typically include loss and accuracy
loss = scores[0]
accuracy = scores[1]

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Loss: 0.7818
Test Accuracy: 72.71%


**BERT**

In [ ]:
df_bert = pd.read_json('/content/drive/MyDrive/Cell_Phones_and_Accessories_5.json', lines='TRUE', nrows = 20000)

In [ ]:
# Step 1: Map Ratings to Sentiments
def map_ratings_to_sentiments(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'

In [ ]:
df_bert['sentiment'] = df_bert['overall'].apply(map_ratings_to_sentiments)
summaries = df_bert['summary'].tolist()
sentiments = df_bert['sentiment'].tolist()

In [ ]:
# Use LabelEncoder to encode sentiment labels as integers
label_encoder = LabelEncoder()
encoded_sentiments = label_encoder.fit_transform(df_bert['sentiment'])

summaries = df_bert['summary'].tolist()
sentiments = encoded_sentiments.tolist()

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
summary_enc = tokenizer_bert(list(summaries), truncation=True, padding=True)

X_summary_input_ids = np.array(summary_enc['input_ids'])
X_summary_attention_mask = np.array(summary_enc['attention_mask'])

In [ ]:
num_classes = df_bert['sentiment'].nunique()  # Number of unique sentiments (negative, neutral, positive)
model_bert_sentiment = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_bert_sentiment.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history_bert_sentiment = model_bert_sentiment.fit(
    {'input_ids': X_summary_input_ids, 'attention_mask': X_summary_attention_mask}, np.array(sentiments),
    validation_split=0.2,
    epochs=5,
    steps_per_epoch = 25,
    batch_size=32)

Epoch 1/5
25/25 [==============================] - 84s 1s/step - loss: 2.4480 - acc: 0.6500 - val_loss: 1.0986 - val_acc: 0.6982
Epoch 2/5
25/25 [==============================] - 20s 795ms/step - loss: 1.0986 - acc: 0.7225 - val_loss: 1.0986 - val_acc: 0.6982
Epoch 3/5
25/25 [==============================] - 29s 1s/step - loss: 1.0986 - acc: 0.7000 - val_loss: 1.0986 - val_acc: 0.6982
Epoch 4/5
25/25 [==============================] - 28s 1s/step - loss: 1.0986 - acc: 0.7300 - val_loss: 1.0986 - val_acc: 0.6982
Epoch 5/5
25/25 [==============================] - 27s 1s/step - loss: 1.0986 - acc: 0.7475 - val_loss: 1.0986 - val_acc: 0.6982


**Conclusion**:

1. We started with simple data cleaning by removing special charecters, punctuations
2. Post which we applied TF-IDF for numerical representation of data
3. Models like logistic regression,naive bayes ,svc and Random forest was used
4. We tried word embeddings and pretrained Glove Embedding
5. We tried to pre-process the target column with positive,neutral and negative sentiments
6. Models like LSTM,RNN and BERT was used to evaluate the accuracy and we observed an increase in accuracy with this technique.